In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("mid").getOrCreate()
sc = spark.sparkContext

22/05/07 13:09:08 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.108.128 instead (on interface ens33)
22/05/07 13:09:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/07 13:09:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import random

### 1

In [23]:
def max_min_sum_count(partition):
    is_first = True
    sum_ = 0
    count = 0
    for element in partition:
        if(is_first==True):
            max_ = element
            min_ = element
            is_first = False
        else:
            max_ = max(element, max_)
            min_ = min(element, min)
        sum_ += element
        count += 1
    return [(max_, min_, sum_, count)]

def min_max_sum_count_map(number):
    return (number, number, number, 1)

def reducer(pair1, pair2):
    return (
    max(pair1[0], pair2[0]), 
    min(pair1[0], pair2[0]), 
    (pair1[0] + pair2[0]), 
    (pair1[1] + pair2[1])
    )

l = [random.randint(1, 1000) for i in range(100)]
rdd1 = sc.parallelize(l)
final = rdd1.map(min_max_sum_count_map).reduce(lambda a,b: (max(a[0], b[0]), min(a[1],b[1]), (a[2]+b[2]), (a[3] + b[3])))
final
#final = rdd.mapPartitions(max_min_sum_count).reduce(lambda a, b: (max(a[0], b[0]), min(a[1], b[1]), sum(a[2], b[2]), sum(a[3], b[3])))
#print("max:" final[0])
#print("min:" final[1])
#print("min:" final[2]/final[3])

(994, 3, 47260, 100)

### 2

In [3]:
def zpn(partition):
    z, p, n = 0, 0, 0
    for element in partition:
        if(element>0):
            p+=1
        elif(element <0):
            n+=1
        else:
            z+=1
    return [(z,p,n)]
    

### 3

In [5]:
'''
def map(key, value):
    emit(key, (value, 1))

def reduce(key, value): key->key (values list, list of 1s) -> value
    emit(sum(value[0] / sum(value[1])))
'''

'\ndef map(key, value):\n    emit(key, (value, 1))\n\ndef reduce(key, value): key->key (value, 1) -> value\n    emit(sum(value[0] / sum(value[1])))\n'

### 4

In [ ]:
'''
distinct()

union()
'''

### 5

In [ ]:
'''

'''

### 6

In [ ]:
'''
joined frame:

b1, c1, NaN
b1, c2, NaN
b2, a2, NaN
b7, c1, a1
b7, c2, a2


def map(record):
    rec = record.split(',')
    key = rec[1]
    value = rec[2]
    emit(key, value)

def reduce(key: 'b', values: 'a' or 'c'):
    


'''

### 8

In [4]:
id_Grades = []
import random
for i in range(100):
    id_Grades.append(str(random.randint(1, 10)) + ',' + str(random.randint(1, 100)))

In [7]:

def mapper(record):
    rec = record.split(',')
    name = rec[0]
    grade = int(rec[1])

    return (name, grade)

def select_min_max(values):
    return (min(values), max(values))


rdd8 = sc.parallelize(id_Grades)
result = rdd8.map(mapper).groupByKey().mapValues(select_min_max).collect()
result

[('4', (8, 90)),
 ('9', (50, 100)),
 ('10', (14, 100)),
 ('1', (5, 92)),
 ('8', (3, 97)),
 ('2', (5, 89)),
 ('5', (1, 96)),
 ('3', (13, 92)),
 ('7', (10, 79)),
 ('6', (2, 88))]

### 11

In [14]:
rdd11 = sc.textFile('11.txt')
def mapper(record):
    words = record.split(',')
    pairs_list = []
    for i in range(len(words) - 1):
        pairs_list.append((words[i], words[i+1]))
    return pairs_list
#rdd11.collect()
tmp = rdd11.flatMap(mapper).map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b).collect()
tmp

[(('you', 'doing'), 2),
 (('Hello', 'how'), 3),
 (('how', 'are'), 4),
 (('are', 'you'), 4),
 (('Hi', 'how'), 1)]

### 12

#### a

In [16]:
'''
("k1", 4)
(k2, 2)
(k3, 2)
(k1, 2)
(k2, 2)
(k2, 2)
(k3, 8)
(k3, 6)
'''

'\n(k1, 4)\n(k2, 2)\n(k3, 2)\n(k1, 2)\n(k2, 2)\n(k2, 2)\n(k3, 8)\n(k3, 6)\n'

#### b

In [17]:
'''
"k1" : [4, 2]
"k2" : [2, 2, 2]
"k3" : [2, 8, 6]
'''

'\n"k1" : [4, 2]\n"k2" : [2, 2, 2]\n"k3" : [2, 8, 6]\n'

#### c

In [18]:
'''
(("k1", 6), ("k2", 6), ("k3", 16))
'''

'\n(("k1", 6), ("k2", 6), ("k3", 16))\n'

### 13 ?

### 14

In [36]:
rdd14 = sc.textFile("16")
#rdd14.getNumPartitions()
#print(rdd14.collect())
def mapper(sentence):
    words = sentence.split(' ')
    cnt = 0
    pcnt = 0
    for word in words:
        cnt+=1
        length = len(word)-1
        is_p = True
        for i in range(len(word)):
            if(word[i] != word[length-i]):
                is_p=False
                break
        if(is_p == True):
            pcnt+=1
    return (pcnt, cnt)

def reducer(pair1, pair2):
    return (pair1[0]+pair2[0], pair1[1] + pair2[1])

#pair = mapper('wow i thought i did forgot all the stuff you told me')
#pair
result = rdd14.map(mapper).reduce(lambda a,b: (a[0]+b[0], a[1]+b[1]))
print(result[0]/result[1])


0.3333333333333333


### 17


### 18
+ a: format of the output of map: `("key", value),("key", key), ("key1",1)/("key2", 2)`
+ b: ("key", []) ("key1", []) ("key2", [])
+ c: 3
+ d: pass


### 19?

In [ ]:
'''
def mapper(key: documentNumber, value: document):
    words = document.split(' ')
    for i in range(len(words)):
        emit(words[i], (key, i))

def reducer(key:word, values: (documentNumber, i)):
    dic = {}
    for element in values:
        if(element[0] not in dic.keys()):
            dic[element[0]] = []
        else:
            dic[element[0]].append(element[1])
    emit(key, [(element: dic[element]) for element in dic.keys()])
'''
'''
#another method
def mapper(key:docNumber, value:doc):
    words = doc.split(' ')
    for i in range(len(words)):
        emit((words[i], key), i)

def reducer1(key:(word,docNumber), values: indexes):
    emit(word, (docNumber, indexes))

def reducer2(key: word, values: (docNumber, indexes)):
    emit(word, (docNumber, indexes))


'''

### 23

In [20]:
rdd23 = sc.textFile('23.txt')
#a
#rdd23.count()
#b
rdd23.map(lambda x: x.split(','))\
.filter(lambda x: x[1]=='1')\
.map(lambda x: (x[0], (int(x[1]), float(x[2]))))\
.reduceByKey(lambda a, b: ((a[0]+ b[0]), (a[1] + b[1])))\
.collect()\

[('g1', (2, 6.3999999999999995)), ('g2', (3, 8.5))]

In [22]:
#c
rdd23.map(lambda x: x.split(','))\
.filter(lambda x: x[1]!='3')\
.collect()

[['g1', '1', '2.3'],
 ['g1', '2', '1.5'],
 ['g1', '1', '4.1'],
 ['g2', '1', '1.3'],
 ['g2', '2', '1.8'],
 ['g2', '1', '4.3'],
 ['g2', '1', '2.9']]

### 24

In [ ]:
#a
'''
(1,1)
(2,2)
(3,1)
(1,3)
(4,4)
(2,1)
(4,2)
(5,5)
'''

#b
'''
1, (1, 3)
2, (1, 2)
3, (1)
4, (2, 4)
5, (5)
'''

### 26?
what is the meaning of 'efficient' in this question?

### 42?

In [31]:
#create a data file
'''
with open('42.txt', 'w') as file:
    for i in range(100):
        file.writelines('user' + str(random.randint(1,10)) + ',movie' + str(random.randint(1,20)) + ',' + str(random.randint(1,5)) + '\n')
'''

rdd42 = sc.textFile('42.txt')

# MUST use the provided functions.
# how to use these fuctions in pyspark program
def getUser(record):
    seg = record.split(',')
    return seg[0]

def getMovie(record):
    return record.split(',')[1]

def getRating(record):
    return record.split(',')[2]
# a 
def generateMovieUsrPairs(record):
    return (getMovie(record), getUser(record))

def generateUsrMoviePairs(record):
    return (getUser(record), getMovie(record))
# a
print(rdd42.map(generateMovieUsrPairs).groupByKey().mapValues(lambda X: len(set(X))).collect())
print('+++++++++++++++++++++++')
#b
print(rdd42.map(generateUsrMoviePairs).groupByKey().mapValues(lambda X: len(set(X))).collect())

[('movie10', 3), ('movie16', 5), ('movie4', 2), ('movie18', 3), ('movie8', 3), ('movie1', 2), ('movie11', 5), ('movie15', 2), ('movie6', 7), ('movie2', 5), ('movie13', 4), ('movie14', 3), ('movie7', 5), ('movie12', 8), ('movie3', 2), ('movie20', 4), ('movie9', 4), ('movie5', 5), ('movie19', 5), ('movie17', 1)]
+++++++++++++++++++++++
[('user6', 7), ('user2', 11), ('user8', 8), ('user5', 7), ('user4', 5), ('user1', 6), ('user9', 7), ('user10', 10), ('user3', 8), ('user7', 9)]


### 47

In [ ]:
#a
'''
def map(key: alphabet, value: list of numbers):
    emit(key, max(value)) # return key and local maximum
'''

#b
'''
a, 50
a, 60
b, 50
b, 55
'''
#c
'''
def reduce(key: alphabet, value : list of local maximums):
    emit(key, max(value)) # global maximum
'''
#d
'''
a, (50, 60)
b, (50, 55)
'''
#e
'''
2
'''

#f
'''
a, 50
b, 55
'''

### 50

In [36]:
data = ["abc", "abc", "xyz", "xyz", "xyz"]
rdd50 = sc.parallelize(data)

rdd50.map(lambda x: (x, 1))\
.reduceByKey(lambda a, b: (a + b))\
.map(lambda x: x[0]).collect()

['abc', 'xyz']

### 54

In [39]:
def myfunc(n):
    if n < 0:
        return [n, -n, -n]
    else:
        return []


data = [0, 1, 2, -3, -4]
rdd54 = spark.sparkContext.parallelize(data)

rdd54.flatMap(myfunc).flatMap(myfunc).collect()

[-3, 3, 3, -4, 4, 4]

### 55 ?

In [40]:
'''
def map(key: <key-as-string>, value: <value-as-int>):
    emit(key, value)


def reduce(key: <key-as-string>, value: list of values with same key):
    value = sorted(value)
    if len(value) % 2==0:
        emit(key, mean(value[int(len(value)/2)], value[int(len(value)/2) + 1]))
    else:
        emit(key, value[int(len(value)/2) + 1])
'''

'\ndef map(key: <key-as-string>, value: <value-as-int>):\n\n'

### 66

In [41]:
data = [('a', 2), ('b', 3), ('d', 2),('x', 3), ('y', 1)]
rdd66 = sc.parallelize(data)

def dup(pair):
    return [pair[0] for i in range(pair[1])]

rdd66.flatMap(dup).collect()

['a', 'a', 'b', 'b', 'b', 'd', 'd', 'x', 'x', 'x', 'y']

### 69

In [42]:
'''
E
'''

'\nE\n'

### 71 ?

### 73?

In [48]:
# what does 'efficient' mean?
data = [1, 11, -1, 2, 12, 3, -4, 13, 4, 14]
rdd73 = sc.parallelize(data)
def countOddEven(number):
    if((number >0) and (number % 2 ==0)):
        return (0, 1)
    elif((number > 0) and (number % 2 != 0)):
        return (1, 0)
    else:
        return (0, 0)

rdd73.map(countOddEven).reduce(lambda x, y: ((x[0] + y[0]), (x[1] + y[1])))

(4, 4)

In [24]:
spark.stop()